### United Nation: Rapid Assessment Tool project 
- Split text document into sentences
- Design a matching scoring algorithm to assign weight to sentences

In [1]:
import pandas as pd

In [2]:
bhutan = pd.read_csv('bhutan_input.csv',index_col=0)

In [3]:
bhutan

,sector,text
0,﻿5.1,Education Sector\n\nThe single most important ...
1,5.2,Health Sector\n\nAll Bhutanese avail free heal...
2,5.3,Human Resource Development & Management\n\nDev...
3,5.4,Employment\n\nThe Labour and Employment Act 20...
4,5.5,Renewable Natural Resources (RNR) Sector\n\nRN...
5,5.6,Tourism Sector\n\nTourism in Bhutan is governe...
6,5.7,"Trade, Industries & Mines Sector\n\nThis is a ..."
7,5.8,"Hydropower, Renewable Energy and Hydromet Sect..."
8,5.9,"Information, Communication & Technology and Me..."
9,5.10,Transport Sector\n\nTransport sector covers s...


In [4]:
para_dict = {}
def sperate(row):
    text = row['text']
    lst = text.split('\n\n')
    sentences = []
    for p in lst:
        sentences = sentences+p.split('. ')
    
    for i,sentence in enumerate(sentences):
        key = str(str(row['sector']).split('.')[-1])+'_'+str(i)
        para_dict[key]=sentence

    return row

In [5]:
para = bhutan.apply(sperate,axis=1)

In [6]:
para = pd.DataFrame(para_dict.values(),index = para_dict.keys(),columns=['p'])

In [7]:
import re
def process(row):
    
    s = row['p']
    s= s.decode('ascii','ignore').encode('ascii','ignore')
    s = s.replace('\t','').replace('\n','')
    s = re.sub(r'^v?i+\)','',s)
    s = s.strip()
    row['p'] = s
    
    return row

In [8]:
para = para.apply(process,axis=1)

In [10]:
para.to_csv('sentences.csv')

## SDG importing

In [41]:
target = pd.read_excel('SDG_Features0420.xlsx')
target.head()

,Number,SDG sub Goal content,"F1_Name,S/W",F1_Dictionary,"F2_Name,S/W",F2_Dictionary,"F3_Name,S/W",F3_Dictionary,"F4_Name,S/W",F4_Dictionary
0,1.1,"By 2030, eradicate extreme povert yfor all peo...","extreme poverty,S","extreme poverty, absolute poverty, destitution...","poverty,W","poverty,poor,famine,poornes,impoverishment,poo...","living cost, W","aliveness,bread and butter,dwell,endure,experi...",NaN,NaN
1,1.2,"By 2030, reduce at least by half the proportio...","proportion,W","portion, proportion, rate","poverty, S","poverty,poor,need,famine,poornes,impoverishmen...","reduce,S","reduce, lessen, make less, make smaller, lower...",NaN,NaN
2,1.3,Implement nationally appropriate social protec...,"social protection,S","social protection, labour markets, capacity, u...","poverty, S","poverty,poor,need,famine,poornes,impoverishmen...","coverage,W","coverage, analysis, insurance, report, broadca...",NaN,NaN
3,1.4,"By 2030, ensure that all men and women, in par...","vulnerables,S","vulnerable, unemployment, ill health, disabili...","equal right, W","equal rights, feminism, rights of women, siste...","economic resource, S","economic resources, goods, services, valuable ...","service,W","service, resource, utilization, distribution, ..."
4,1.5,"By 2030, build the resilience of the poor and ...","vulnerables,S","unemployment, ill health, disability, work-rel...","extreme event,S","extreme event\n climate-related, extreme event...","resilience, W","elasticity, flexibility, pliancy, recoil, snap...",NaN,NaN


In [42]:
target = target.fillna(0)

In [36]:
para = pd.read_csv('sentences.csv')

In [37]:
## preprocess tokens 
## n-grams
## to lower
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

porter = nltk.PorterStemmer()

def get_ngrams(text, n ):
    n_grams = ngrams(word_tokenize(text), n)
    return [ ' '.join(grams) for grams in n_grams]

def pre(row):
    text = row['p']
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    
    tokens = tokens + get_ngrams(text,2)
    tokens= tokens + get_ngrams(text,3)
    tokens= tokens + get_ngrams(text,4)
    
    tokens = [porter.stem(w) for w in tokens]
    row['tokens'] = tokens
    return row

In [38]:
para = para.apply(pre,axis=1)

##  matching 

In [30]:
import pandas as pd
import numpy as np
para = pd.read_csv('sentences.csv')

In [27]:
target = pd.read_excel('SDG_Features0420.xlsx')
target = target.fillna(0)

In [43]:
target.columns = ['target','content','F1','F1D','F2','F2D','F3','F3D','F4','F4D']

In [44]:
target['target']=target.target.astype('str')

In [45]:
target.ix[105,'target']

'16.1'

In [46]:
#change target 16.10 into string "16.10", originally identify as 16.1
target.ix[105,'target']='16.10'

In [48]:
# run 4 times 
# 1st time: range(0,31)
# 2nd time: range(31,60)
# 3rd time: range(60,90)
# 4th time: range(90,107)
import nltk
porter = nltk.PorterStemmer()
for t_i in range(0,106):
    def func(row):
        tokens = row['tokens']
        ###
        #s = 'poverty Human Settlement - To ensure sustainable development of human settlements, new legislation and policies will be formulated and existing rules and regulations reviewed and improved. Development of valley plans, regional hub plans and rural urban development plans will be accelerated in order to contain the rural urban migration. Capacity strengthening of engineers, architects and planners will be given emphasis.'
        #tokens = s.split(' ')
        ###
        
        s1,s2,s3,s4 = 1,1,1,1
        w1,w2,w3,w4 = 0,0,0,0
        
        flag = 0 
        
        if target.ix[t_i,'F1']==0: #isnan 
            s1=0 #missing for the entire target
        else:
            for word in target.ix[t_i,'F1D'].split(','):
                if porter.stem(word.strip()) in tokens:
                    if target.ix[t_i,'F1'].split(',')[-1].strip()=='S':
                        flag = 1 
                    else: # 'W'
                        w1 = 0.1 
            if (flag==0) & (target.ix[t_i,'F1'].split(',')[-1].strip() =='S'): #is strong feature, and not match 
                s1 = 0 
                    
        flag = 0
        if target.ix[t_i,'F2']==0: #isnan 
            s2=1
        else:
            for word in target.ix[t_i,'F2D'].split(','):
                if porter.stem(word.strip()) in tokens:
                    if target.ix[t_i,'F2'].split(',')[-1].strip() =='S':
                        flag = 1 
                    else: # 'W'
                        w2 = 0.1 
            if (flag ==0) & (target.ix[t_i,'F2'].split(',')[-1].strip() =='S'): #is strong feature, and not match 
                s2 = 0
        
        
        flag = 0
        if target.ix[t_i,'F3']==0:
            s3=1
        else:
            for word in target.ix[t_i,'F3D'].split(','):
                if porter.stem(word.strip()) in tokens:
                    if target.ix[t_i,'F3'].split(',')[-1].strip() =='S':
                        flag = 1 
                    else: # 'W'
                        w3 = 0.1
                        
            if (flag ==0) & (target.ix[t_i,'F3'].split(',')[-1].strip() =='S'): #is strong feature, and not match 
                s3 = 0
                    
        flag = 0
        if target.ix[t_i,'F4']==0:
            s4=1
        else:
            for word in target.ix[t_i,'F4D'].split(','):
                if porter.stem(word.strip()) in tokens:
                    if target.ix[t_i,'F4'].split(',')[-1].strip() =='S':
                        flag = 1 
                    else: # 'W'
                        w4 = 0.1  
            if (flag ==0) & (target.ix[t_i,'F4'].split(',')[-1].strip() =='S'): #is strong feature, and not match 
                s4 = 0
            
            
        score = s1*s2*s3*s4*(1+w1+w2+w3+w4)
        
        
        row[str(target.ix[t_i,'target'])] = score 
        
        return row
    
    para = para.apply(func,axis=1)

In [49]:
para=para.rename(columns={'Unnamed: 0':'sector_sent'})

In [51]:
def splt(row):
    row['sector'] = str(row['sector_sent']).split('_')[0]
    row['sent_id'] = str(row['sector_sent']).split('_')[-1]
    return row

In [52]:
para=para.apply(splt,axis=1)

In [53]:
para['sector'] = para.sector.astype('int')
para['sent_id'] = para.sent_id.astype('int')
para = para.set_index(['sector','sent_id'])
para = para.sort_index()

In [55]:
para = para.drop(['sector_sent','tokens'],axis=1)

In [56]:
para.to_excel('sentence_result0421.xlsx')